# Code Sandbox

A place to test new code and functions

In [1]:
# import packages and set working directory
import numpy as np
import matplotlib.pyplot as plt
import os
from math import exp
#os.chdir('../maize-Toff')

# import objects
from farm import Climate
from farm import Soil
from farm import Crop
from farm import CropModel

In [2]:
climate = Climate() # uses default climate values
soil = Soil('loam')
crop = Crop(soil=soil) # previously kc_max=1.2, LAI_max=2.0, T_max=4.0
soil.set_nZr(crop)  
model = CropModel(crop=crop,soil=soil,climate=climate)



In [3]:
soil.calc_t_sfc(0.8,E_max=5)

ImportError: cannot import name 'ln' from 'math' (/Users/kellycaylor/opt/anaconda3/envs/maize-Toff/lib/python3.8/lib-dynload/math.cpython-38-darwin.so)